# 🦸🏻 Understanding Semantic Kernel with Heroes 🦸🏼‍♀️

Semantic Kernel has been created to allow developers to seamlessly integrate Artificial Intelligence into their applications. To achieve this, it provides a set of features that will enable you to add models, prompts, native functions, and memories without requiring deep knowledge of AI 🥲. That's why it's said that Semantic Kernel simulates the brain 🧠 of your application.

<div style="text-align:center">
    <img src="images/semantic-kernel.png" width="15%">
</div>

## (Optional) - Creating an Azure Open AI 🤖 Service and Deployments

Before diving into Semantic Kernel 🛝, you'll need to have one of the supported services set up. Currently, you can choose from the following options: [Azure Open AI](https://azure.microsoft.com/en-us/products/ai-services/openai-service), [Open AI](https://openai.com/), or [Hugging Face](https://huggingface.co/).

In this example, I am going to use Azure Open AI.

Therefore, through Azure CLI, I need to log in:

In [ ]:
az login

If you need it, because you have many tenants and subscriptions under your account, you can select the one that interests you (or has access to Azure Open AI 😊) through the following command:


In [ ]:
az account set -n "Visual Studio Enterprise Subscription"

Now, to be able to create what you need for this notebook, set the following variables with your preferred values:


In [ ]:
$RESOURCE_GROUP="understanding-semantic-kernel"
$LOCATION="canadaeast"
$AZURE_OPEN_AI="ai-for-heroes"

With them, you can now create the resource group:


In [ ]:
az group create `
--name $RESOURCE_GROUP `
--location $LOCATION

An Azure Open AI resource:


In [ ]:
az cognitiveservices account create `
--name $AZURE_OPEN_AI `
--custom-domain $AZURE_OPEN_AI `
--resource-group $RESOURCE_GROUP `
--kind OpenAI `
--sku S0 `
--location $LOCATION

And finally, you need a deployment of any of the models you have available. In this example, I am going to use gpt-4:


In [ ]:
az cognitiveservices account deployment create `
--name $AZURE_OPEN_AI `
--resource-group $RESOURCE_GROUP `
--deployment-name "gpt-4" `
--model-name "gpt-4" `
--model-version "0613"  `
--model-format OpenAI `
--sku-capacity "10" `
--sku-name "Standard"

The **sku-capacity** parameter allows us to specify how many tokens per minute we can send to this model. To see how your quota usage is, you can use this other command:


In [ ]:
az cognitiveservices usage list `
-l $LOCATION

Load enviroment variables with your Azure Open AI endpoint and key

In [ ]:
$env:AZURE_OPEN_AI_KEY =$(az cognitiveservices account keys list `
--name $AZURE_OPEN_AI `
--resource-group $RESOURCE_GROUP `
--query "key1" `
--output tsv)

$env:AZURE_OPEN_AI_ENDPOINT =$(az cognitiveservices account show `
--name $AZURE_OPEN_AI `
--resource-group $RESOURCE_GROUP `
--query "properties.endpoint" `
--output tsv)

#dir env:AZURE_OPEN_AI_KEY
#dir env:AZURE_OPEN_AI_ENDPOINT

## How to get started with Semantic Kernel


The first thing you need to be able to run Semantic Kernel in this notebook is to install the **Microsoft.SemanticKernel** library, which is currently in version **1.5.0** (I promise to keep updating 🤓). 

You can find more information about this library and its updates on the official NuGet page [here](https://www.nuget.org/packages/Microsoft.SemanticKernel).



In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.5.0"

Installed Packages Microsoft.SemanticKernel, 1.5.0

With it, you can now instantiate the brain of your application through **KernelBuilder**. It has a lot of connectors to the models and other things, but for now, let's start with the basics:


In [8]:
using Microsoft.SemanticKernel;

// Create a Builder
var builder = Kernel.CreateBuilder();

Depending on the type of task you want to do, you can use different methods with the prefix **With**. For now, we are going to use some of the models of type **completion** or completion.


In [9]:
// OpenAI keys
var modelId = "gpt-4";
var apiKey = (await Microsoft.DotNet.Interactive.Kernel.GetPasswordAsync("Give me your Open AI key")).GetClearTextPassword();

// Create a chat completion service
builder.AddOpenAIChatCompletion(modelId, apiKey);

In [ ]:
// Azure OpenAI keys
var deploymentName = "gpt-4";
var endpoint = Environment.GetEnvironmentVariable("AZURE_OPEN_AI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("AZURE_OPEN_AI_KEY");

// Create a chat completion service
builder.AddAzureOpenAIChatCompletion(deploymentName, endpoint, apiKey);

With the configuration done, all that remains is to generate the kernel with everything set:


In [10]:
var kernel = builder.Build();

## The plugins

Plugins are the core of Semantic Kernel. With them, you encapsulate capabilities so that they can be reusable, maintainable, and plannable (you will understand it later 🙃). There are two types: those that consist of prompt templates called **Semantic Functions** and native functions of the chosen programming language called **Native Functions**.

<div style="text-align:center">
    <img src="images/writer-plugin-example.png" width="40%" />
</div>

### Semantic Functions

When you talk to artificial intelligence models, you must do so with what is known as a *prompt*. This can range from a simple phrase to something more elaborate, allowing the model to understand not only what we want but also how we want it. If you take a look at the official documentation, it defines this type of function as the mouth 👄 and ears 👂🏻 of your brain 🧠. 🤖🌐

<div style="text-align:center">
    <img src="images/semantic-function-explainer.png" width="20%" />
</div>

As part of this repo, you have a folder called **SemanticFunctions** that has different functions of this type:

- **FunPlugin**: This allows us to ask the model to make jokes about heroes under certain conditions, through the **Joke** function.
- **WritePlugin**: To show you that within a plugin you can have different functions, in this directory we have two related to the art of writing: the first one, **OOF**, allows us to generate the "Out of Office" 🏢📧 message for superheroes and the second **StoryGen** will help us create stories, also about superheroes 🦸🏻‍♂️🦸🏻‍♀️.

For our kernel to know that these plugins are available, you first need to get the path of the directory:


In [11]:
using System.IO;

var pluginsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "SemanticFunctions", "FunPlugIn");

And then start loading the plugins you want:


In [12]:
// Load the FunPlugin from the Plugins Directory
var funPluginFunctions = kernel.ImportPluginFromPromptDirectory(pluginsDirectory);

To be able to invoke a function of this type, you can do it in the following way:


In [13]:
var joke = new KernelArguments() { ["input"] = "tell me a joke" };
var result = await kernel.InvokeAsync(funPluginFunctions["Joke"], joke);

As you can see, just by using **kernel.RunAsync** and passing my request as parameters and which function, in this case **Joke**, within the plugin, in this case **FunPlugin**, I want to use.

Each of these functions consists of two files:

- **skprompt.txt** is the file where the prompt to send to the model is defined in a template format, so that it can receive parameters and make them more reusable.

```
WRITE EXACTLY ONE JOKE or HUMOROUS STORY ABOUT THE SUBJECT BELOW

JOKE MUST BE:
- IN ENGLISH

BE CREATIVE AND FUNNY. I WANT TO LAUGH.

Incorporate the hero if provided: {{$hero}}
+++++

{{$input}}
+++++
```
- **config.json**, which allows us to indicate the maximum number of tokens allowed for this call (**max_tokens**), the temperature to control the randomness of the responses (**temperature**), which means that closer to 1 they will be more random and closer to zero more determined and focused on the most likely response, **top_p** is used to control the diversity of the responses, where a value of 0.0 means that only the most likely responses will be considered and 1 where all possible responses will be considered and **presence_penalty** and **frequency_penalty** to adjust the penalty for the presence and frequency of the tokens in the generated responses. On the other hand, if the skprompt.txt file receives parameters, they must also be defined in this file in the **parameters** array of the **input** object.

```javascript
{
  "schema": 1,
  "description": "Generate a funny joke about heroes",
  "models": [
    {
      "max_tokens": 150,
      "temperature": 0.9,
      "top_p": 0.5,
      "presence_penalty": 0.2,
      "frequency_penalty": 0.3
    }
  ],
  "input": {
    "parameters": [
      {
        "name": "input",
        "description": "Joke subject",
        "defaultValue": ""
      },
      {
        "name": "hero",
        "description": "Give a hint about the hero you want to joke about",
        "defaultValue": ""
      }
    ]
  }
}
```


To see the result generated from calling this Joke function, you can retrieve it in the following way:


In [14]:
Console.WriteLine(result.GetValue<string>());

Sure, here's a joke for you:

Why don't superheroes like to play hide and seek with The Invisible Man?

Because he always wins, even when they use their x-ray vision!


As you can see, in this execution we have only passed the input parameter, but not the hero about whom we wanted to generate this joke. To be able to send more than one parameter you need to create an object of the **ContextVariables** type:


In [15]:
var variables = new KernelArguments() {
    ["input"] = "Tell me a Christmas joke",
    ["hero"] = "Ironman"
};

To be able to use these two values as part of the call, simply put it as the first argument.


In [16]:
var result = await kernel.InvokeAsync(funPluginFunctions["Joke"], variables);

Now check if the joke is about the hero specified as a parameter.


In [17]:
Console.WriteLine(result.GetValue<string>());

Why did Ironman decorate the Christmas tree with tiny repulsor beams?

Because he wanted to make sure his Christmas was lit... but also well-defended from Ultron!


In the same way, we can use the functions included in **WriterPlugin**:


In [18]:
// Load the WriterPlugin from the Plugins Directory
var pluginsDirectoryWriter = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "SemanticFunctions", "WriterPlugin");

var writerPluginFunctions = kernel.ImportPluginFromPromptDirectory(pluginsDirectoryWriter);


One that generates stories:


In [21]:
var variables = new KernelArguments() {
    ["input"] = "Tell me a Christmas Story",
    ["hero"] = "Ironman"
};

var result = await kernel.InvokeAsync(writerPluginFunctions["StoryGen"], variables);
Console.WriteLine(result);

Una vez, en una fría noche de Navidad 🎄❄️, nuestro héroe Ironman, también conocido como Tony Stark, volaba por los cielos 🌌 vigilando la tranquila ciudad de Nueva York. 

A mitad de noche 🌛, escuchó un leve sonido 🎵 que provenía del callejón. Descendió para investigar y encontró a un niño llorando 😢. El niño, asustado por el frío y la oscuridad, había perdido a su madre en la multitud de las fiestas navideñas.

Ironman, con su corazón heroico, decidió ayudar. Levantó al niño en sus brazos y con su traje luminoso ✨ voló sobre la ciudad en busca de su madre. Desde el cielo 🌠, el niño se sintió maravillado por las luces brillantes de la ciudad y su miedo se convirtió en admiración 😮.

Finalmente, encontraron a la madre del niño en un parque. Al ver a su hijo a salvo y sano, la madre lloró de felicidad 😭. Le agradeció a Ironman, quien le sonrió bajo su casco de hierro.

Antes de irse, Ironman dejó un regalo 🎁 debajo del árbol de Navidad para el niño. Luego, se elevó en el cielo y la noche 

Or even to create messages for when the heroes 🦸🏻‍♀️ are on vacation ✈️🚢🌴:


In [22]:
var variables = new KernelArguments() {
    ["input"] = "Create an OOF for Christmas",
    ["hero"] = "Hulk"
};

var result = await kernel.InvokeAsync(writerPluginFunctions["OOF"], variables);
Console.WriteLine(result.GetValue<string>());

Error: Microsoft.SemanticKernel.KernelFunctionCanceledException: The invocation of function 'OOF' was canceled.
 ---> System.Threading.Tasks.TaskCanceledException: The operation was cancelled because it exceeded the configured timeout of 0:01:40. Network timeout can be adjusted in ClientOptions.Retry.NetworkTimeout.
 ---> System.Threading.Tasks.TaskCanceledException: The operation was canceled.
 ---> System.Threading.Tasks.TaskCanceledException: The operation was canceled.
 ---> System.IO.IOException: Unable to read data from the transport connection: The I/O operation has been aborted because of either a thread exit or an application request..
 ---> System.Net.Sockets.SocketException (995): The I/O operation has been aborted because of either a thread exit or an application request.
   --- End of inner exception stack trace ---
   at System.Net.Sockets.Socket.AwaitableSocketAsyncEventArgs.ThrowException(SocketError error, CancellationToken cancellationToken)
   at System.Net.S

## Native Functions

While semantic functions allow us to define and reuse prompts, **with native functions you can make the semantic kernel call functions written in C# or Python**, for tasks that go beyond a call through a prompt.

<div style="text-align:center">
    <img src="images/native-function-explainer.png" width="25%">
</div>

### Why do I need native functions in these types of applications?

Large language models (LLMs) are excellent for generating text, but there are several tasks they cannot perform on their own. These include, among others:

- Retrieving data from external data sources
- Knowing what time it is
- Performing complex mathematical operations
- Completing tasks in the real world
- Memorizing and remembering information

For these scenarios, and many others, native functions are very useful 👍🏻


For this example, I'm going to use an API called **SuperHero API**, which requires an API key. You can get it from their website: [https://superheroapi.com/](https://superheroapi.com/)

Once you have it, pass it to the prompt that appears with the following line:


In [27]:
var superHeroApiKey = (await Microsoft.DotNet.Interactive.Kernel.GetPasswordAsync("Give me your Super Hero Api key")).GetClearTextPassword();

Now, to load a native function, we must add this before we build the Kernel.

Let's create a new Super Hero Info class and add it as a Plugin.

In [28]:
#load "NativeFunctions/HeroInfo.cs"

var heroInfo = new HeroInfo(superHeroApiKey);
builder.Plugins.AddFromObject(heroInfo, "HeroInfo");
Kernel kernel = builder.Build();

The builder now can interact with the local function. Let's make a call to get the alter ego of a super hero.

In [32]:
var kernelArgs = new KernelArguments()
{
    ["input"] = "Ironman"
};

var result = await kernel.InvokeAsync<string>("HeroInfo", "GetAlterEgo", kernelArgs);
Console.WriteLine(result);

Tony Stark


In this repo, there is another directory called **NativeFunctions** where you can find a class called **HeroInfo.cs**. In it, there is a function decorated with the **KernelFunction** attribute, which allows us to indicate to the Semantic Kernel that it is a native function and, through the **Description** property, also give it information about what the purpose of this function is. In this case, what this method allows us to do is retrieve the alter ego of the superhero that we pass as a parameter, in this example that of Ironman. 


## Planner

So far, all the plugins you have seen have been executed intentionally. That is, no one has chosen them for you and you can run them based on your needs. However, this is the most *static* way to interact with Semantic Kernel. There is another option called **Planner** that will leave you astounded 😮

Planner is a function that takes a user's request and returns a plan on how to carry out the request. To do this, it uses AI to combine the plugins registered in the core and recombine them into a series of steps that complete a goal.

<div style="text-align:center;">
    <img src="images/the-planner.png" width=35% />
</div>

To see it in action, we are going to use the plugins that you already know.

The first thing you need is to add the nuget reference to the Planner Handlebars.

To learn more about this NuGet package, you can visit [here](https://www.nuget.org/packages/Microsoft.SemanticKernel.Planners.Handlebars/1.5.0-preview#show-readme-container).



In [33]:
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.5.0-preview"

Installed Packages Microsoft.SemanticKernel.Planners.Handlebars, 1.5.0-preview

Now we are ready to create a planner.

In [34]:
using Microsoft.SemanticKernel.Planning.Handlebars;

#pragma warning disable SKEXP0060
var planner = new HandlebarsPlanner();


There are different types as you can see [here](https://learn.microsoft.com/en-us/semantic-kernel/ai-orchestration/planners/?tabs=Csharp). In this example, we are going to use the one that executes tasks sequentially.

Since you already have all the plugins loaded in your kernel instance, we can ask it something like this:


In [36]:
using System.Text.Json;

var ask = "I would like you to tell me a joke about Batman, and with that joke, create an out-of-office message using the joke.";
#pragma warning disable SKEXP0060
var originalPlan = await planner.CreatePlanAsync(kernel, ask);

Console.WriteLine("Original plan:\n");
Console.WriteLine(originalPlan);

Original plan:

{
  "Prompt": "\u003Csystem~\u003E## Instructions\nExplain how to achieve the user\u0027s goal using the available helpers with a Handlebars .Net template.\n\n## Example\nIf the user posed the goal below, you could answer with the following template.\u003C/system~\u003E\n\u003Cuser~\u003E## Goal\nI want you to generate 10 random numbers and send them to another helper.\u003C/user~\u003E\n\u003Cassistant~\u003EHere\u0027s a Handlebars template that achieves the goal:\n\u0060\u0060\u0060handlebars\n{{!-- Step 0: Extract key values --}}\n{{set\n  \u0022count\u0022\n  10\n}}\n{{!-- Step 1: Loop using the count --}}\n{{#each\n  (range\n    1\n    count\n  )\n}}\n  {{!-- Step 2: Create random number --}}\n  {{set\n    \u0022randomNumber\u0022\n    (Example-Random\n      seed=this\n    )\n  }}\n  {{!-- Step 3: Call example helper with random number and print the result to the screen --}}\n  {{set\n    \u0022result\u0022\n    (Example-Helper\n      input=randomNumber\n    )\n  

As you can see, the planner knows about the  plugins thanks to the description that I included as part of their implementation.

Time to run the plan and get an OOF for **Batman** that includes a **bad joke**.


In [38]:
// executing the plan
#pragma warning disable SKEXP0060
var originalPlanResult = await originalPlan.InvokeAsync(kernel, new KernelArguments());

Console.WriteLine("Original Plan results:\n");
Console.WriteLine(originalPlanResult.ToString());

Original Plan results:

I am currently out of office. Here's a joke to lighten your day: Why did Batman go to the store? Because he ran out of bat-teries!


## LABS

The kernel already has the plugins loaded, and the function, let's see if it can solve a more complex example:


In [44]:
var ask_complex = "I would like you to find out IronMan's alter ego, and create an out of office message for IronMan, signing it with his alter ego.";

#pragma warning disable SKEXP0060
var plan_complex = await planner.CreatePlanAsync(kernel, ask_complex);

Console.WriteLine("Plan:\n");
Console.WriteLine(plan_complex);

Plan:

{{!-- Step 1: Get alter ego for IronMan --}}
{{#with (HeroInfo-GetAlterEgo 'IronMan') as |ironManAlterEgo|}}
  {{!-- Step 2: Get alter ego for Batman --}}
  {{#with (HeroInfo-GetAlterEgo 'Batman') as |batmanAlterEgo|}}
    {{!-- Step 3: Set the out of office message --}}
    {{set 'message' (concat 'Batman is currently out of office. Please contact ' batmanAlterEgo ' for any urgent matters.')}}

    {{!-- Step 4: Output the alter ego of IronMan and the message --}}
    {{json (concat 'IronMan is also known as ' ironManAlterEgo '. ' 'Out of Office Message: ' message)}}
  {{/with}}
{{/with}}


Let's execute the plan and see its output:


In [42]:
#pragma warning disable SKEXP0060
var complexPlanResult = await plan_complex.InvokeAsync(kernel, new KernelArguments());

Console.WriteLine("Complex  Plan results:\n");
Console.WriteLine(complexPlanResult.ToString());

Complex  Plan results:

Batman is currently out of office. Signed, Terry McGinnis


## Kernel Memory

<img src="images/How kernel memory works.png" width="80%" />

For this example, I'm going to use **Open AI** instead of Azure Open AI, so you need to save [an API Key](https://platform.openai.com/account/api-keys) from this in the following variable:


To be able to use Kernel Memory, you need to add its nuget library, in addition to importing the class that I have generated in the **KernelMemory** directory.

To learn more about the Kernel Memory library, you can visit the [NuGet package page](https://www.nuget.org/packages/Microsoft.KernelMemory.Core).


In [ ]:
#r "nuget: Microsoft.KernelMemory.Core, 0.11.231120.6-preview"

#!import "KernelMemory/Memories.cs"

The first thing I'm going to do is import a few *memories* (texts) and *documents* into Kernel Memory.


In [ ]:
var openApiKey = await Microsoft.DotNet.Interactive.Kernel.GetPasswordAsync("Give me your Open AI key");

MemoryKernel.Init(openApiKey.GetClearTextPassword());

As you can see in the output, it already takes care of generating the embeddings of the sentences/documents that we pass to it so that the GPT-4 model can generate the answer.
Now that we have some content to ask about, let's load this class as another plugin.


In [ ]:
var memoriesPlugin = kernel.ImportFunctions(new MemoryKernel(), "MemoriesPlugin");

And now let's ask about the content:


In [ ]:
var planner = new SequentialPlanner(kernel);

var plan = await planner.CreatePlanAsync("Who is Bruno's favorite hero?");

var result = await kernel.RunAsync(plan);

Console.WriteLine(result.GetValue<string>());

In [ ]:
var planner = new SequentialPlanner(kernel);

var plan = await planner.CreatePlanAsync("What was the last movie Gisela saw?");

var result = await kernel.RunAsync(plan);

Console.WriteLine(result.GetValue<string>());

We can also ask about the PDF that I included:


In [ ]:
var planner = new SequentialPlanner(kernel);

var plan = await planner.CreatePlanAsync("¿Qué incluye este volumen de batman?");

var result = await kernel.RunAsync(plan);

Console.WriteLine(result.GetValue<string>());